In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats





df = pd.read_csv("..\\data\\benin-malanville.csv")
#print(df.head(10))
#print(df.describe())



In [ ]:
print("Description of the numerical columns:")
print(df.describe())
print("Description of the categorical columns:")
print(df.describe(include=['object']))
print("Description of all columns:")
print(df.describe(include='all'))
print("An overview of the dataset:")
print(df.head())
print("The number of rows and columns in the dataset:")
print(df.shape)
print("An overview of the dataset:")
print(df.tail())
print("An overview of the dataset: 5 random rows")
print(df.sample(5))
print("An overview of the dataset: the shape of the dataset")
print(df.info())
print("Check for missing values:")
print(df.isna().sum())
print("Check for duplicates:")
print(df.duplicated().sum())


In [21]:
print("An overview of the dataset: 5 random rows")
print(df.sample(5))

An overview of the dataset: 5 random rows
               Timestamp    GHI    DNI    DHI   ModA   ModB  Tamb    RH   WS  \
308488  2022-03-11 05:29   -5.4   -0.4   -7.7    0.0    0.0  27.7  25.9  2.7   
357006  2022-04-13 22:07   -1.9   -0.4   -2.1    0.0    0.0  34.2  40.6  6.3   
295830  2022-03-02 10:31  736.6  527.5  326.4  755.5  751.3  33.2  12.9  1.1   
310143  2022-03-12 09:04  212.2    1.6  207.7  199.2  189.2  29.2  28.1  1.4   
325136  2022-03-22 18:57   -4.2   -1.0   -6.3    6.5    5.4  38.5  12.1  1.7   

        WSgust  WSstdev     WD  WDstdev   BP  Cleaning  Precipitation  TModA  \
308488     3.1      0.4   50.8      7.9  990         0            0.0   26.2   
357006     7.9      1.0  244.8     11.6  990         0            0.0   34.6   
295830     1.9      0.5  176.8     12.4  994         0            0.0   66.0   
310143     1.9      0.4   56.1     10.5  993         0            0.0   37.8   
325136     1.9      0.3  312.6      2.3  989         0            0.0   36.8 